-- CVRP with CPLEX:

-- volume max is included

-- sales min is included

-- max duration for each truck is included 

# Capacitated Vehicle Routing Problem

<img src="http://neo.lcc.uma.es/dynamic/images/vrp.png" width="400" />

- $n$ is the number of clientes
- $N$ is set of clients, with $N=\{1,2,...,n\}$
- $V$ is set of vetices (or nodes), with $V=\{0\}\cup N$
- $A$ is set of arcs, with $A=\{(i,j)\in V^2 : i\neq j\}$
- $c_{ij}$ is cost of travel over arc $(i,j)\in A$
- $Q$ is the vehicle capacity
- $q_i$ is the amount that has to be delivered to customer $i\in N$

Then, the formulation is the following
$$\begin{align}
\min \quad & \sum_{i,j\in A} c_{ij} x_{ij} \\
\text{s.t.} \quad & \sum_{j\in V , j\neq i} x_{ij} = 1  && i \in N \\
& \sum_{i\in V , i\neq j} x_{ij} = 1 && j \in N \\
& \text{if} \ x_{ij}=1 \ \Rightarrow \ u_i + q_j = u_j && i,j \in A : j\neq 0, i\neq 0 \\
& q_i \leq u_i \leq Q && i \in N \\
& x_{ij} \in \{0,1\} && i,j \in A
\end{align}$$

https://www.youtube.com/watch?v=-hGL39jdtQE

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import sys
try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install docplex
    else:
        !pip install --user docplex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Install and Import packages

In [ ]:
!pip install geopandas
!pip install pandas_bokeh
!pip install matplotlib
!pip install plotly_express
!pip install osmnx

# !pip uninstall rtree
!pip install libspatialindex-dev
!pip install rtree
!pip install pyyaml==5.4.14

!pip install geopy
!pip install mlrose
!pip install six

!pip install ortools

!pip install pulp
!pip install gmaps
!pip install googlemaps 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.5.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.3
    Uninstalling matplotlib-3.1.3:
      Successfully uninstalled matplotlib-3.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement libspatialindex-dev (from versions: none)
ERROR: No matching distribution found for libspatialindex-dev
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pyyaml==5.4.14 (from versions: 3.10, 3.11, 3.12, 3.13b1, 3.13rc1, 3.13, 4.2b1, 4.2b2, 4.2b4, 5.1b1, 5.1b3, 5.1b5, 5.1, 5.1.1, 5.1.2, 5.2b1, 5.2, 5.3b1, 5.3, 5.3.1, 5.4b1, 5.4b2, 5.4, 5.4.1, 6.0b1, 6.0)
ERROR: No matching distribution found for pyyaml==5.4.14
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in index

In [ ]:
# you may run into this problem https://stackoverflow.com/questions/61867945/python-import-error-cannot-import-name-six-from-sklearn-externals
# I have manually updated site-packages/mlrose/neural.py chaning 
# original:
# from sklearn.external import six
# to:

import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose # for travelling salesman problem

import pandas as pd
import numpy as np
import numpy as nph

!pip install matplotlib==3.1.3
import matplotlib.pyplot as plt


from geopy import distance
import requests # to call the openmap/google apis
import json
import datetime
import math
import itertools

import plotly_express as px
# px.__version__
import geopandas as gpd
from shapely.geometry import Point, LineString


import networkx as nx
import osmnx as ox
ox.config(use_cache=True, log_console=True)

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import numpy as np
import pandas as pd
import pulp
import itertools

import googlemaps
import gmaps


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.1.3-cp37-cp37m-manylinux1_x86_64.whl (13.1 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.2
    Uninstalling matplotlib-3.5.2:
      Successfully uninstalled matplotlib-3.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
osmnx 1.1.2 requires matplotlib>=3.4, but you have matplotlib 3.1.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import geopandas
from gmaps.geotraitlets import Longitude

data_path = "/content/drive/MyDrive/geo/data/"
work_path = "/content/drive/MyDrive/geo/Route_Simulation/CVRP_Romeoville/"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Cleaning

In [ ]:
date = "12_20_2021" # enter the date as 'mm_dd_yyy'

In [ ]:
All_count_clusters_year = pd.read_csv(f"{data_path}/Romeoville/All_count_clusters_yearly_to_compare.csv") 
five_clusters_2021 = All_count_clusters_year[["zipcode","latitude", "longitude", "2021_clusters_count_avg_5", "RouteCodeID"]]
print(five_clusters_2021.head())

md = pd.read_csv(f"{data_path}/Romeoville/December_20_2021/delivery_report_Chicago_{date}.csv")  
md.rename(columns = {"Zip Code":"zipcode"}, inplace = True)

zips = pd.read_csv(f"{data_path}/Romeoville/USZIPCodeToCityStateDB.csv")
zips = zips.rename(columns = {"Zip Code": "zipcode", "ZipLatitude":"latitude", "ZipLongitude":"longitude"})
zips = zips.drop_duplicates(subset = "zipcode",keep='first')
zips = zips.dropna()
print("\n=============== Zipcode geographical data ===============\n\n")
print(zips.head())

md_df = pd.merge(md, five_clusters_2021, on = "zipcode")
print("\n=============== Delivery Data for the day with geographical data ===============\n\n")
print(md_df.head())
print("\n=============== Columns of Delivery Data for the day with geographical data ===============\n\n")
print(md_df.columns)

md_df = md_df[['Route#', 'Stop#', 'Address', 'zipcode', 'latitude', 'longitude','Amount Delivered', 'Volume Delivered', 'Distance (m)', 'RouteCodeID','2021_clusters_count_avg_5']]
print("\n=============== Delivery Data for the day with geographical data with required columns===============\n\n")
print(md_df)

md_df = md_df[["Route#","Stop#","Address","zipcode",'latitude','longitude','Amount Delivered', 'Volume Delivered', 'Distance (m)','RouteCodeID', '2021_clusters_count_avg_5']]
md_df = md_df.drop_duplicates(subset = ["Route#","Stop#","Address"])
print("\n=============== Delivery Data (Cleaned) ===============\n\n")
print(md_df)

to_drop = md_df.loc[(md_df["zipcode"] == 60446) & (md_df["Volume Delivered"].isnull())] # when the volume is 0 in 60446, it means that it is the depot.
print("drop ", to_drop.index, " since they are the depot entries"  )

md_df.drop(to_drop.index, axis=0, inplace=True)
print("\n=============== Delivery Data ready for the analysis ===============\n\n")
print(md_df)

   zipcode   latitude  longitude  2021_clusters_count_avg_5 RouteCodeID
0    46001  40.239110 -85.673842                          4       ROM07
1    46011  40.113328 -85.767502                          0       ROM07
2    46012  40.154339 -85.625772                          0       ROM07
3    46013  40.050362 -85.686448                          0       ROM07
4    46016  40.101240 -85.672430                          4       ROM07

=============== Zipcode geographical data ===============


   zipcode        City     County State  CountyFIPS  StateFIPS  TimeZone  \
0      501  HOLTSVILLE    SUFFOLK    NY         103         36         5   
1      544  HOLTSVILLE    SUFFOLK    NY         103         36         5   
2      601    ADJUNTAS   ADJUNTAS    PR           1         72         4   
3      602      AGUADA     AGUADA    PR           3         72         4   
4      603       RAMEY  AGUADILLA    PR           5         72         4   

  DayLightSavings   latitude  longitude  
0       

# Data Cleaning - end

In [ ]:
md_df

,Route#,Stop#,Address,zipcode,latitude,longitude,Amount Delivered,Volume Delivered,Distance (m),RouteCodeID,2021_clusters_count_avg_5
0,c01,1,5502 DANBURRY DRV,61109,42.194732,-89.063558,1799.99,137.00,87.780,ROM01,3
1,c01,2,4229 PINECREST RD,61107,42.284408,-89.009781,4299.00,0.00,5.320,ROM01,1
2,c02,5,916 SCOTTSWOOD RD,61107,42.284408,-89.009781,1634.98,138.00,2.843,ROM01,1
3,c01,3,3223 MONTLAKE DR,61114,42.308048,-88.986835,1172.98,55.63,3.380,ROM01,3
4,c01,4,4685 HIGH POINT DR 20,61114,42.308048,-88.986835,1489.99,47.20,3.770,ROM01,3
...,...,...,...,...,...,...,...,...,...,...,...
652,romca,40,14005 PARAMOUNT WAY,46303,41.373628,-87.464439,0.00,31.77,10.200,ROM01,3
654,romca,44,3855 E 56TH STREET,46220,39.866058,-86.101651,0.00,67.75,8.030,ROM07,3
655,romca,45,509 WOODDRUFF PL E DR,46201,39.775737,-86.109626,0.00,53.00,6.870,ROM07,2
656,romca,47,1700 GREENBRIAR DR,60914,41.190774,-87.855861,0.00,54.00,150.920,ROM01,0


# EDA

In [ ]:
print("\n=============== Truck Codes for each route - 01 and 07 ===============\n\n")
print(md_df["Route#"][md_df["RouteCodeID"] == "ROM01"].unique(), md_df["Route#"][md_df["RouteCodeID"] == "ROM07"].unique())

rom01_df = md_df[md_df["RouteCodeID"] == "ROM01"] 
rom01_df_sums = rom01_df.groupby(['Route#']).sum()
rom01_df_sums = rom01_df_sums[['Amount Delivered','Volume Delivered','Distance (m)']]
print("\nROM01 - Total delivery values for each truck:\n\n", rom01_df_sums)

rom07_df = md_df[md_df["RouteCodeID"] == "ROM07"] 
rom07_df_sums = rom07_df.groupby(['Route#']).sum()
rom07_df_sums = rom01_df_sums[['Amount Delivered','Volume Delivered','Distance (m)']]
print("\nROM07 - Total delivery values for each truck:\n\n", rom07_df_sums)

for i in range(0,5):
    print("\n========================= Cluster: ", i, "=========================")
    temp_df = md_df[md_df["2021_clusters_count_avg_5"] == i] 
    temp_df_sums = temp_df.groupby(['Route#']).sum()
    print(temp_df_sums)




=============== Truck Codes for each route - 01 and 07 ===============


['c01' 'c02' 'c12' 'romca' 'c03' 'c07' 'c04' 'c05' 'c06' 'c13' 'c08' 'c09'
 'c10' 'c16' 'c24' 'c11' 'c14' 'c15' 'c22' 'c17' 'c21' 'c18' 'c19' 'c20'
 'c27' 'c23' 'c25' 'c26'] ['c56' 'c57' 'c63' 'c58' 'c59' 'c60' 'c62' 'c61' 'romca' 'c64' 'c65']

ROM01 - Total delivery values for each truck:

         Amount Delivered  Volume Delivered  Distance (m)
Route#                                                  
c01             14727.53            578.87       141.161
c02             13760.69            747.41       201.222
c03             15375.73            824.58       128.733
c04             20638.78           1130.37       170.957
c05             18377.57           1116.33       157.989
c06             21839.62           1154.63       112.049
c07             15947.93           1327.88        87.034
c08             14313.30            960.45       200.163
c09             18020.87           1006.45       150.949
c10   

# EDA - end

# Functions for Routing 

In [ ]:
#### COST(Distance/Duration Calculator) ####

def get_distance(point1: dict, point2: dict) -> tuple:
    """Gets distance between two points en route using http://project-osrm.org/docs/v5.10.0/api/#nearest-service"""
    ''' durations array of arrays that stores the matrix in row-major order. durations[i][j] gives the travel time from the i-th waypoint to the j-th waypoint. 
    Values are given in seconds. Can be null if no route between i and j can be found. '''

    '''distance: The distance, in metres, between each pair of coordinates '''
    url = f"""http://router.project-osrm.org/route/v1/driving/{point1["longitude"]},{point1["latitude"]};{point2["longitude"]},{point2["latitude"]}?overview=false&alternatives=false"""
    r = requests.get(url)
    
    # get the distance from the returned values
    route = json.loads(r.content)["routes"][0]
    return (route["distance"], route["duration"])

# Functions for Routing - end

# Prepare data for a single truck routing

In [ ]:
rom07 = md_df["Route#"][md_df["RouteCodeID"] == "ROM07"].unique()
rom01 = md_df["Route#"][md_df["RouteCodeID"] == "ROM01"].unique()
rom07, rom01

(array(['c56', 'c57', 'c63', 'c58', 'c59', 'c60', 'c62', 'c61', 'romca',
        'c64', 'c65'], dtype=object),
 array(['c01', 'c02', 'c12', 'romca', 'c03', 'c07', 'c04', 'c05', 'c06',
        'c13', 'c08', 'c09', 'c10', 'c16', 'c24', 'c11', 'c14', 'c15',
        'c22', 'c17', 'c21', 'c18', 'c19', 'c20', 'c27', 'c23', 'c25',
        'c26'], dtype=object))

In [ ]:
truck_code= "c58"
df = md_df[["Route#", "Stop#", "zipcode", "latitude", "longitude", "Amount Delivered", "Volume Delivered", 'Distance (m)']]
truck_df = df[df["Route#"] == truck_code]
truck_df = truck_df.reset_index()
print(f"\n ================ Data for {truck_code} =================\n")
print(truck_df)

print(f"\n ==== Total deilvered numbers for {truck_code} ==== \n")
print(" Total volume delivered:", truck_df["Volume Delivered"].sum(),"\n Total amount delivered", truck_df["Amount Delivered"].sum(), "\n Total distance delivered",  truck_df["Distance (m)"].sum())

df_vol = truck_df[["latitude", "longitude", "Volume Delivered"]]
print(f"\n ==== Deilvered volume to each zipcode for {truck_code} ==== \n")
print("\n", df_vol)

print(f"\n ==== Create an empty array for distance matrix ==== \n")
_distance_result = np.zeros((len(truck_df),len(truck_df)))
print(_distance_result)


 ================ Data for c58 =================

   index Route#  Stop#  zipcode   latitude  longitude  Amount Delivered  \
0    580    c58      1    46077  39.982864 -86.288475           4649.94   
1    581    c58      2    46077  39.982864 -86.288475            349.99   
2    582    c58      3    46268  39.896883 -86.232390            299.99   
3    583    c58      4    46268  39.896883 -86.232390           3039.97   
4    584    c58      5    46268  39.896883 -86.232390           2139.99   
5    585    c58      6    46268  39.896883 -86.232390              0.00   
6    586    c58      7    46260  39.894530 -86.181154              0.00   

   Volume Delivered  Distance (m)  
0            229.35       179.240  
1              0.00         6.502  
2              7.55         5.370  
3             38.28         2.110  
4             58.25         1.720  
5             16.11         1.847  
6            104.30         2.560  

 ==== Total deilvered numbers for c58 ==== 

 Total volume 

In [ ]:
truck_df = truck_df.reset_index()
truck_df.drop("index", axis=1, inplace=True)
truck_df = truck_df.rename(columns = {"level_0":"index"})
truck_df = truck_df[["index", "zipcode", "latitude", "longitude", "Amount Delivered", "Volume Delivered"]]
truck_df = truck_df.rename(columns = {"Amount Delivered":"sales", "Volume Delivered":"volume"})

depot_zipcode = "60446"
depot_latitude = "41.636022"
depot_longitude = "-88.095961"

truck_df.loc[len(truck_df)] = [-1,depot_zipcode,depot_latitude,depot_longitude,0,0] # add depot as the row 0
truck_df = truck_df.sort_values(by = "index")
truck_df = truck_df.reset_index(drop = True)
truck_df = truck_df.reset_index()
truck_df.drop("index", axis=1, inplace = True)
truck_df = truck_df.rename(columns = {"level_0":"index "})
print(f"\n ============== {truck_code} Data ready for analysis ============== \n")
print(truck_df)


 ============== c58 Data ready for analysis ============== 

   index  zipcode   latitude   longitude    sales  volume
0       0   60446  41.636022  -88.095961     0.00    0.00
1       1   46077  39.982864  -86.288475  4649.94  229.35
2       2   46077  39.982864  -86.288475   349.99    0.00
3       3   46268  39.896883   -86.23239   299.99    7.55
4       4   46268  39.896883   -86.23239  3039.97   38.28
5       5   46268  39.896883   -86.23239  2139.99   58.25
6       6   46268  39.896883   -86.23239     0.00   16.11
7       7   46260   39.89453  -86.181154     0.00  104.30


# Prepare data for a single truck routing - End

# Compute Distance and Duration

In [ ]:
truck_df

,index,zipcode,latitude,longitude,sales,volume
0,0,60446,41.636022,-88.095961,0.00,0.00
1,1,46077,39.982864,-86.288475,4649.94,229.35
2,2,46077,39.982864,-86.288475,349.99,0.00
3,3,46268,39.896883,-86.23239,299.99,7.55
4,4,46268,39.896883,-86.23239,3039.97,38.28
5,5,46268,39.896883,-86.23239,2139.99,58.25
6,6,46268,39.896883,-86.23239,0.00,16.11
7,7,46260,39.89453,-86.181154,0.00,104.30


In [ ]:
ce_cities = truck_df[["zipcode",'latitude','longitude']]
# get the distance and duration from OSRM API for all our combinations.
print(ce_cities)

dist_array = []
dist_cost = []
dur_cost = []
temp_df = truck_df.copy()

for i , r in ce_cities.iterrows():
    print("i=",i,"r=",r)
    point1 = {"latitude": r["latitude"], "longitude": r["longitude"]}
    for j, o in ce_cities[ce_cities.index != i].iterrows():
        point2 = {"latitude": o["latitude"], "longitude": o["longitude"]}
        dist, duration = get_distance(point1, point2)
        #dist = geodesic((i_lat, i_lon), (o["CapitalLatitude"], o["CapitalLongitude"])).km
        
        dist_array.append((temp_df.iloc[i,0], temp_df.iloc[j,0], duration*0.0166667, dist*0.000621371)) #temp_df.iloc[i,0], temp_df.iloc[j,0]
        # dist_array.append((temp_df.iloc[i,0], temp_df.iloc[j,0], duration*0.0166667, dist)) #temp_df.iloc[i,0], temp_df.iloc[j,0]
        dist_cost.append((temp_df.iloc[i,0], temp_df.iloc[j,0], dist*0.000621371)) #temp_df.iloc[i,0], temp_df.iloc[j,0]  dist*0.000621371
        dur_cost.append((temp_df.iloc[i,0], temp_df.iloc[j,0], duration*0.0166667)) #temp_df.iloc[i,0], temp_df.iloc[j,0]


print("Distance Cost (minutes and miles)",dist_array)   

  zipcode   latitude   longitude
0   60446  41.636022  -88.095961
1   46077  39.982864  -86.288475
2   46077  39.982864  -86.288475
3   46268  39.896883   -86.23239
4   46268  39.896883   -86.23239
5   46268  39.896883   -86.23239
6   46268  39.896883   -86.23239
7   46260   39.89453  -86.181154
i= 0 r= zipcode           60446
latitude      41.636022
longitude    -88.095961
Name: 0, dtype: object
i= 1 r= zipcode          46077
latitude     39.982864
longitude   -86.288475
Name: 1, dtype: object
i= 2 r= zipcode          46077
latitude     39.982864
longitude   -86.288475
Name: 2, dtype: object
i= 3 r= zipcode          46268
latitude     39.896883
longitude    -86.23239
Name: 3, dtype: object
i= 4 r= zipcode          46268
latitude     39.896883
longitude    -86.23239
Name: 4, dtype: object
i= 5 r= zipcode          46268
latitude     39.896883
longitude    -86.23239
Name: 5, dtype: object
i= 6 r= zipcode          46268
latitude     39.896883
longitude    -86.23239
Name: 6, dtype: object


In [ ]:
dist_array

[(0, 1, 220.63877460999998, 182.4045755178),
 (0, 2, 220.63877460999998, 182.4045755178),
 (0, 3, 220.5171077, 186.25813204850002),
 (0, 4, 220.5171077, 186.25813204850002),
 (0, 5, 220.5171077, 186.25813204850002),
 (0, 6, 220.5171077, 186.25813204850002),
 (0, 7, 223.56544713, 187.96093713690001),
 (1, 0, 218.96043792, 182.087800582),
 (1, 2, 0.0, 0.0),
 (1, 3, 19.45670558, 9.3223048388),
 (1, 4, 19.45670558, 9.3223048388),
 (1, 5, 19.45670558, 9.3223048388),
 (1, 6, 19.45670558, 9.3223048388),
 (1, 7, 23.1500463, 11.7796407325),
 (2, 0, 218.96043792, 182.087800582),
 (2, 1, 0.0, 0.0),
 (2, 3, 19.45670558, 9.3223048388),
 (2, 4, 19.45670558, 9.3223048388),
 (2, 5, 19.45670558, 9.3223048388),
 (2, 6, 19.45670558, 9.3223048388),
 (2, 7, 23.1500463, 11.7796407325),
 (3, 0, 219.10877155, 186.08470740240003),
 (3, 1, 19.53670574, 9.3003704425),
 (3, 2, 19.53670574, 9.3003704425),
 (3, 4, 0.0, 0.0),
 (3, 5, 0.0, 0.0),
 (3, 6, 0.0, 0.0),
 (3, 7, 6.14834563, 2.9015540216000004),
 (4, 0, 219.

In [ ]:
# distance
c_dist= {}
for l2 in dist_array:
    x = c_dist[l2[:2]] = l2[3]
# print(c_dist)

distance = pd.DataFrame(columns = range(0,len(df_vol)+1), index = range(0,len(df_vol)+1))
dist_df = pd.DataFrame(dist_cost)
dist_df = dist_df.reset_index()
print("Distance DataFrame: \n\n",dist_df)

for i in range(len(df_vol)+1):
    for j in range(len(df_vol)+1):
        if i==j:
            distance.iloc[i,j] = 0
        else:
            # print("i:",i,"j:",j, "index:", x_df["index"].loc[(x_df[0] == i) & (x_df[1] == j)])
            a = dist_df["index"].loc[(dist_df[0] == i) & (dist_df[1] == j)]
            distance.iloc[i,j] = float(dist_df.iloc[a,3].values)

print("Distance matrix",distance)

Distance DataFrame: 

     index  0  1           2
0       0  0  1  182.404576
1       1  0  2  182.404576
2       2  0  3  186.258132
3       3  0  4  186.258132
4       4  0  5  186.258132
5       5  0  6  186.258132
6       6  0  7  187.960937
7       7  1  0  182.087801
8       8  1  2    0.000000
9       9  1  3    9.322305
10     10  1  4    9.322305
11     11  1  5    9.322305
12     12  1  6    9.322305
13     13  1  7   11.779641
14     14  2  0  182.087801
15     15  2  1    0.000000
16     16  2  3    9.322305
17     17  2  4    9.322305
18     18  2  5    9.322305
19     19  2  6    9.322305
20     20  2  7   11.779641
21     21  3  0  186.084707
22     22  3  1    9.300370
23     23  3  2    9.300370
24     24  3  4    0.000000
25     25  3  5    0.000000
26     26  3  6    0.000000
27     27  3  7    2.901554
28     28  4  0  186.084707
29     29  4  1    9.300370
30     30  4  2    9.300370
31     31  4  3    0.000000
32     32  4  5    0.000000
33     33  4  6    0.0000

In [ ]:
truck_df

,index,zipcode,latitude,longitude,sales,volume
0,0,60446,41.636022,-88.095961,0.00,0.00
1,1,46077,39.982864,-86.288475,4649.94,229.35
2,2,46077,39.982864,-86.288475,349.99,0.00
3,3,46268,39.896883,-86.23239,299.99,7.55
4,4,46268,39.896883,-86.23239,3039.97,38.28
5,5,46268,39.896883,-86.23239,2139.99,58.25
6,6,46268,39.896883,-86.23239,0.00,16.11
7,7,46260,39.89453,-86.181154,0.00,104.30


In [ ]:
distance

,0,1,2,3,4,5,6,7
0,0,182.404576,182.404576,186.258132,186.258132,186.258132,186.258132,187.960937
1,182.087801,0,0.0,9.322305,9.322305,9.322305,9.322305,11.779641
2,182.087801,0.0,0,9.322305,9.322305,9.322305,9.322305,11.779641
3,186.084707,9.30037,9.30037,0,0.0,0.0,0.0,2.901554
4,186.084707,9.30037,9.30037,0.0,0,0.0,0.0,2.901554
5,186.084707,9.30037,9.30037,0.0,0.0,0,0.0,2.901554
6,186.084707,9.30037,9.30037,0.0,0.0,0.0,0,2.901554
7,187.73451,11.779206,11.779206,2.901554,2.901554,2.901554,2.901554,0


In [ ]:
# duration
c_dur = {}
for l2 in dist_array:
    x = c_dur[l2[:2]] = l2[2]

# print(c_dur)

duration = pd.DataFrame(columns = range(0,len(df_vol)+1), index = range(0,len(df_vol)+1))
dur_df = pd.DataFrame(dur_cost)
dur_df = dur_df.reset_index()
print(dur_df)

for i in range(len(df_vol)+1):
    for j in range(len(df_vol)+1):
        if i==j:
            duration.iloc[i,j] = 0
        else:
            # print("i:",i,"j:",j, "index:", x_df["index"].loc[(x_df[0] == i) & (x_df[1] == j)])
            a = dur_df["index"].loc[(dur_df[0] == i) & (dur_df[1] == j)]
            duration.iloc[i,j] = float(dur_df.iloc[a,3].values)

    index  0  1           2
0       0  0  1  220.638775
1       1  0  2  220.638775
2       2  0  3  220.517108
3       3  0  4  220.517108
4       4  0  5  220.517108
5       5  0  6  220.517108
6       6  0  7  223.565447
7       7  1  0  218.960438
8       8  1  2    0.000000
9       9  1  3   19.456706
10     10  1  4   19.456706
11     11  1  5   19.456706
12     12  1  6   19.456706
13     13  1  7   23.150046
14     14  2  0  218.960438
15     15  2  1    0.000000
16     16  2  3   19.456706
17     17  2  4   19.456706
18     18  2  5   19.456706
19     19  2  6   19.456706
20     20  2  7   23.150046
21     21  3  0  219.108772
22     22  3  1   19.536706
23     23  3  2   19.536706
24     24  3  4    0.000000
25     25  3  5    0.000000
26     26  3  6    0.000000
27     27  3  7    6.148346
28     28  4  0  219.108772
29     29  4  1   19.536706
30     30  4  2   19.536706
31     31  4  3    0.000000
32     32  4  5    0.000000
33     33  4  6    0.000000
34     34  4  7    6

In [ ]:
cds

NameError: ignored

# Compute Distance and Duration - end

# Route with Pulp

In [ ]:
retail_capacity = 1
vehicle_capacity = 1250 #df.volume.sum() + 1
vehicle_count = 3
customer_count = len(truck_df)


In [ ]:
# solve with pulp
for vehicle_count in range(1,vehicle_count+1):
    
    # definition of LpProblem instance
    problem = pulp.LpProblem("CVRP", pulp.LpMinimize)

    # definition of variables which are 0/1
    x = [[[pulp.LpVariable("x%s_%s,%s"%(i,j,k), cat="Binary") if i != j else None for k in range(vehicle_count)]for j in range(customer_count)] for i in range(customer_count)]

    # add objective function
    problem += pulp.lpSum(distance[i][j] * x[i][j][k] if i != j else 0
                          for k in range(vehicle_count) 
                          for j in range(customer_count) 
                          for i in range(customer_count))

    # constraints
    # foluma (2)
    for j in range(1, customer_count):
        problem += pulp.lpSum(x[i][j][k] if i != j else 0 
                              for i in range(customer_count) 
                              for k in range(vehicle_count)) == 1 

    # foluma (3)
    for k in range(vehicle_count):
        problem += pulp.lpSum(x[0][j][k] for j in range(1,customer_count)) == 1
        problem += pulp.lpSum(x[i][0][k] for i in range(1,customer_count)) == 1

    # foluma (4)
    for k in range(vehicle_count):
        for j in range(customer_count):
            problem += pulp.lpSum(x[i][j][k] if i != j else 0 
                                  for i in range(customer_count)) -  pulp.lpSum(x[j][i][k] for i in range(customer_count)) == 0

    #foluma (5)
    for k in range(vehicle_count):
        problem += pulp.lpSum(truck_df.volume[j] * x[i][j][k] if i != j else 0 for i in range(customer_count) for j in range (1,customer_count)) <= vehicle_capacity 

    #foluma (5) retail value
    for k in range(vehicle_count):
        problem += pulp.lpSum(truck_df.sales[j] * x[i][j][k] if i != j else 0 for i in range(customer_count) for j in range (1,customer_count)) >= retail_capacity 
   
   # fomula (6)
    subtours = []
    for i in range(2,customer_count):
         subtours += itertools.combinations(range(1,customer_count), i)

    for s in subtours:
        problem += pulp.lpSum(x[i][j][k] if i !=j else 0 for i, j in itertools.permutations(s,2) for k in range(vehicle_count)) <= len(s) - 1

    
    # print vehicle_count which needed for solving problem
    # print calculated minimum distance value
    if problem.solve() != 1:
        print(f"for {vehicle_count} vehicles. there is no optimal solution!")

    else:
        print('Vehicle Requirements:', vehicle_count)
        
        print('Moving Distance:', pulp.value(problem.objective))

        for v in problem.variables():
            if v.varValue>0:
                print(v.name, "=", v.varValue)
        break
        
        # visualization : plotting with matplolib
        plt.figure(figsize=(8,8))
        for i in range(customer_count):    
            if i == 0:
                plt.scatter(truck_df.latitude[i], truck_df.longitude[i], c='green', s=200)
                plt.text(truck_df.latitude[i], truck_df.longitude[i], "depot", fontsize=12)
            else:
                plt.scatter(truck_df.latitude[i], truck_df.longitude[i], c='orange', s=200)
                plt.text(truck_df.latitude[i], truck_df.longitude[i], str(truck_df.zipcode[i]), fontsize=12)

        for k in range(vehicle_count):
            for i in range(customer_count):
                for j in range(customer_count):
                    if i != j and pulp.value(x[i][j][k]) == 1:
                        plt.plot([truck_df.latitude[i], truck_df.latitude[j]], [truck_df.longitude[i], truck_df.longitude[j]], c="black")

        plt.show()

            

# Route with Pulp - end

# Visulaize Route

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

truck_df.latitude = pd.to_numeric(truck_df.latitude)
truck_df.longitude = pd.to_numeric(truck_df.longitude)
type(truck_df.latitude[0])

In [ ]:
# visualization : plotting with matplolib
plt.figure(figsize=(8,8))
for i in range(customer_count):    
    if i == 0:
        plt.scatter(truck_df.latitude[i], truck_df.longitude[i], c='green', s=200)
        plt.text(truck_df.latitude[i], truck_df.longitude[i], "depot", fontsize=12)
    else:
        plt.scatter(truck_df.latitude[i], truck_df.longitude[i], c='orange', s=200)
        plt.text(truck_df.latitude[i], truck_df.longitude[i], str(truck_df.zipcode[i]), fontsize=12)

for k in range(vehicle_count):
    for i in range(customer_count):
        for j in range(customer_count):
            if i != j and pulp.value(x[i][j][k]) == 1:
                plt.plot([truck_df.latitude[i], truck_df.latitude[j]], [truck_df.longitude[i], truck_df.longitude[j]], c="black")

plt.show()

In [ ]:
truck_df

In [ ]:
# visualization : plotting on google maps
fig = gmaps.figure()
layer = []
color_list = ["red","blue","green"]

for k in range(vehicle_count):
    for i in range(customer_count):
        for j in range(customer_count):
            if i != j and pulp.value(x[i][j][k]) == 1:
                layer.append(gmaps.directions.Directions(
                    (truck_df.latitude[i],truck_df.longitude[i]),
                    (truck_df.latitude[j],truck_df.longitude[j]),
                    mode='car',stroke_color=color_list[k],stroke_opacity=1.0, stroke_weight=5.0))
                    

# visualization : plotting with matplolib
plt.figure(figsize=(10,10))
for i in range(customer_count):    
    if i == 0:
        plt.scatter(truck_df.latitude[i], truck_df.longitude[i], c='green', s=200)
        plt.text(truck_df.latitude[i], truck_df.longitude[i], str(truck_df.zipcode[i]), fontsize=12)
    else:
        plt.scatter(truck_df.latitude[i], truck_df.longitude[i], c='orange', s=200)
        plt.text(truck_df.latitude[i], truck_df.longitude[i], str(truck_df.zipcode[i]), fontsize=12)

for k in range(vehicle_count):
    for i in range(customer_count):
        for j in range(customer_count):
            if i != j and pulp.value(x[i][j][k]) == 1:
                plt.plot([truck_df.latitude[i], truck_df.latitude[j]], [truck_df.longitude[i], truck_df.longitude[j]], c="black")

plt.show()

# Visulaize Route - end

In [ ]:
md_df["Distance (m)"][md_df["Route#"] == "c58"].sum()

In [ ]:
rom07 = md_df["Route#"][md_df["RouteCodeID"] == "ROM07"].unique()
rom01 = md_df["Route#"][md_df["RouteCodeID"] == "ROM01"].unique()
rom07, rom01

for truck_code in rom07: 
    print(truck_code)    
    # truck_code= "c58"
    df = md_df[["Route#", "Stop#", "zipcode", "latitude", "longitude", "Amount Delivered", "Volume Delivered", 'Distance (m)']]
    truck_df = df[df["Route#"] == truck_code]

    if len(truck_df) > 10:
        print("too much zipcodes!", len(truck_df))

    else:       
        truck_df = truck_df.reset_index()
        print(f"\n ================ Data for {truck_code} =================\n")
        print(truck_df)

        print(f"\n ==== Total deilvered numbers for {truck_code} ==== \n")
        print(" Total volume delivered:", truck_df["Volume Delivered"].sum(),"\n Total amount delivered", truck_df["Amount Delivered"].sum(), "\n Total distance delivered",  truck_df["Distance (m)"].sum())

        df_vol = truck_df[["latitude", "longitude", "Volume Delivered"]]
        print(f"\n ==== Deilvered volume to each zipcode for {truck_code} ==== \n")
        print("\n", df_vol)

        print(f"\n ==== Create an empty array for distance matrix ==== \n")
        _distance_result = np.zeros((len(truck_df),len(truck_df)))
        print(_distance_result)

        truck_df = truck_df.reset_index()
        truck_df.drop("index", axis=1, inplace=True)
        truck_df = truck_df.rename(columns = {"level_0":"index"})
        truck_df = truck_df[["index", "zipcode", "latitude", "longitude", "Amount Delivered", "Volume Delivered"]]
        truck_df = truck_df.rename(columns = {"Amount Delivered":"sales", "Volume Delivered":"volume"})

        depot_zipcode = "60446"
        depot_latitude = "41.636022"
        depot_longitude = "-88.095961"

        truck_df.loc[len(truck_df)] = [-1,depot_zipcode,depot_latitude,depot_longitude,0,0] # add depot as the row 0
        truck_df = truck_df.sort_values(by = "index")
        truck_df = truck_df.reset_index(drop = True)
        truck_df = truck_df.reset_index()
        truck_df.drop("index", axis=1, inplace = True)
        truck_df = truck_df.rename(columns = {"level_0":"index "})
        print(f"\n ============== {truck_code} Data ready for analysis ============== \n")
        print(truck_df)

        ce_cities = truck_df[["zipcode",'latitude','longitude']]
        # get the distance and duration from OSRM API for all our combinations.
        print(ce_cities)

        dist_array = []
        dist_cost = []
        dur_cost = []
        temp_df = truck_df.copy()

        for i , r in ce_cities.iterrows():
            print("i=",i,"r=",r)
            point1 = {"latitude": r["latitude"], "longitude": r["longitude"]}
            for j, o in ce_cities[ce_cities.index != i].iterrows():
                point2 = {"latitude": o["latitude"], "longitude": o["longitude"]}
                dist, duration = get_distance(point1, point2)
                #dist = geodesic((i_lat, i_lon), (o["CapitalLatitude"], o["CapitalLongitude"])).km
                
                dist_array.append((temp_df.iloc[i,0], temp_df.iloc[j,0], duration*0.0166667, dist*0.000621371)) #temp_df.iloc[i,0], temp_df.iloc[j,0]
                # dist_array.append((temp_df.iloc[i,0], temp_df.iloc[j,0], duration*0.0166667, dist)) #temp_df.iloc[i,0], temp_df.iloc[j,0]
                dist_cost.append((temp_df.iloc[i,0], temp_df.iloc[j,0], dist*0.000621371)) #temp_df.iloc[i,0], temp_df.iloc[j,0]  dist*0.000621371
                dur_cost.append((temp_df.iloc[i,0], temp_df.iloc[j,0], duration*0.0166667)) #temp_df.iloc[i,0], temp_df.iloc[j,0]


        print("Distance Cost (minutes and miles)",dist_array)

        # distance
        c_dist= {}
        for l2 in dist_array:
            x = c_dist[l2[:2]] = l2[3]
        # print(c_dist)

        distance = pd.DataFrame(columns = range(0,len(df_vol)+1), index = range(0,len(df_vol)+1))
        dist_df = pd.DataFrame(dist_cost)
        dist_df = dist_df.reset_index()
        print("Distance DataFrame: \n\n",dist_df)

        for i in range(len(df_vol)+1):
            for j in range(len(df_vol)+1):
                if i==j:
                    distance.iloc[i,j] = 0
                else:
                    # print("i:",i,"j:",j, "index:", x_df["index"].loc[(x_df[0] == i) & (x_df[1] == j)])
                    a = dist_df["index"].loc[(dist_df[0] == i) & (dist_df[1] == j)]
                    distance.iloc[i,j] = float(dist_df.iloc[a,3].values)

        print("Distance matrix",distance)

        # duration
        c_dur = {}
        for l2 in dist_array:
            x = c_dur[l2[:2]] = l2[2]

        # print(c_dur)

        duration = pd.DataFrame(columns = range(0,len(df_vol)+1), index = range(0,len(df_vol)+1))
        dur_df = pd.DataFrame(dur_cost)
        dur_df = dur_df.reset_index()
        print(dur_df)

        for i in range(len(df_vol)+1):
            for j in range(len(df_vol)+1):
                if i==j:
                    duration.iloc[i,j] = 0
                else:
                    # print("i:",i,"j:",j, "index:", x_df["index"].loc[(x_df[0] == i) & (x_df[1] == j)])
                    a = dur_df["index"].loc[(dur_df[0] == i) & (dur_df[1] == j)]
                    duration.iloc[i,j] = float(dur_df.iloc[a,3].values)
        retail_capacity = 1
        vehicle_capacity = 1250 #df.volume.sum() + 1
        vehicle_count = 3
        customer_count = len(truck_df)

        ################ solve with pulp ################
        for vehicle_count in range(1,vehicle_count+1):
            
            # definition of LpProblem instance
            problem = pulp.LpProblem("CVRP", pulp.LpMinimize)

            # definition of variables which are 0/1
            x = [[[pulp.LpVariable("x%s_%s,%s"%(i,j,k), cat="Binary") if i != j else None for k in range(vehicle_count)]for j in range(customer_count)] for i in range(customer_count)]

            # add objective function
            problem += pulp.lpSum(distance[i][j] * x[i][j][k] if i != j else 0
                                for k in range(vehicle_count) 
                                for j in range(customer_count) 
                                for i in range(customer_count))

            # constraints
            # foluma (2)
            for j in range(1, customer_count):
                problem += pulp.lpSum(x[i][j][k] if i != j else 0 
                                    for i in range(customer_count) 
                                    for k in range(vehicle_count)) == 1 

            # foluma (3)
            for k in range(vehicle_count):
                problem += pulp.lpSum(x[0][j][k] for j in range(1,customer_count)) == 1
                problem += pulp.lpSum(x[i][0][k] for i in range(1,customer_count)) == 1

            # foluma (4)
            for k in range(vehicle_count):
                for j in range(customer_count):
                    problem += pulp.lpSum(x[i][j][k] if i != j else 0 
                                        for i in range(customer_count)) -  pulp.lpSum(x[j][i][k] for i in range(customer_count)) == 0

            #foluma (5)
            for k in range(vehicle_count):
                problem += pulp.lpSum(truck_df.volume[j] * x[i][j][k] if i != j else 0 for i in range(customer_count) for j in range (1,customer_count)) <= vehicle_capacity 

            #foluma (5) retail value
            for k in range(vehicle_count):
                problem += pulp.lpSum(truck_df.sales[j] * x[i][j][k] if i != j else 0 for i in range(customer_count) for j in range (1,customer_count)) >= retail_capacity 
        
        # fomula (6)
            subtours = []
            for i in range(2,customer_count):
                subtours += itertools.combinations(range(1,customer_count), i)

            for s in subtours:
                problem += pulp.lpSum(x[i][j][k] if i !=j else 0 for i, j in itertools.permutations(s,2) for k in range(vehicle_count)) <= len(s) - 1

            
            # print vehicle_count which needed for solving problem
            # print calculated minimum distance value
            if problem.solve() != 1:
                print(f"for {vehicle_count} vehicles. there is no optimal solution!")

            else:
                print('Vehicle Requirements:', vehicle_count)
                
                print('Moving Distance:', pulp.value(problem.objective))
                

                for v in problem.variables():
                    if v.varValue>0:
                        print(v.name, "=", v.varValue)
                
                break

               

In [ ]:
# visualization : plotting on google maps
fig = gmaps.figure()
layer = []
color_list = ["red","blue","green"]

for k in range(vehicle_count):
    for i in range(customer_count):
        for j in range(customer_count):
            if i != j and pulp.value(x[i][j][k]) == 1:
                layer.append(gmaps.directions.Directions(
                    (truck_df.latitude[i],truck_df.longitude[i]),
                    (truck_df.latitude[j],truck_df.longitude[j]),
                        mode='car',stroke_color=color_list[k],stroke_opacity=1.0, stroke_weight=5.0))
                    

# visualization : plotting with matplolib
plt.figure(figsize=(10,10))
for i in range(customer_count):    
    if i == 0:
        plt.scatter(truck_df.latitude[i], truck_df.longitude[i], c='green', s=200)
        plt.text(truck_df.latitude[i], truck_df.longitude[i], str(truck_df.zipcode[i]), fontsize=12)
    else:
        plt.scatter(truck_df.latitude[i], truck_df.longitude[i], c='orange', s=200)
        plt.text(truck_df.latitude[i], truck_df.longitude[i], str(truck_df.zipcode[i]), fontsize=12)

for k in range(vehicle_count):
    for i in range(customer_count):
        for j in range(customer_count):
            if i != j and pulp.value(x[i][j][k]) == 1:
                plt.plot([truck_df.latitude[i], truck_df.latitude[j]], [truck_df.longitude[i], truck_df.longitude[j]], c="black")

print(plt.show())
